In [1]:
#importing libraries
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [2]:
#loading the pretrained models for detecting face and to detect wearing mask or not
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface.xml")
mask_detection = tf.keras.models.load_model("face_mask_detection.h5")

In [3]:
#text,style to be displayed
text_mask = "Mask On"
text_no_mask = "Mask Off"
font = cv2.FONT_HERSHEY_SIMPLEX
scale = 0.8

In [16]:
def predict(image):
    #coverting the image to rgb
    face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #resizing the image
    face_frame = cv2.resize(face_frame, (224, 224))
    
    #converting image to array and adding up 4th dimension
    face_frame = img_to_array(face_frame)
    face_frame = np.expand_dims(face_frame, axis=0)
    
    #preprocessing the input
    face_frame =  preprocess_input(face_frame)
    
    #predicting the image whether it is a masked or unmasked label using pretrained model
    prediction = mask_detection.predict(face_frame)
    
    #returning the result
    return prediction[0][0]

In [17]:
# function to detect face using haar cascade model and facemask using trained model and drawing a rectangle,text with respect to the output.
def detector(gray_image, frame):
    
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5)
    
    for (x,y,w,h) in faces:

        roi_color = frame[y:y+h, x:x+w]
        mask = predict(roi_color)

        
        if mask > 0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, text =text_mask, org =(x+50,y-100), fontFace =font, fontScale = scale, color =(0,255,0),
            thickness = 2)
        
        elif mask<=0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
            cv2.putText(frame, text =text_no_mask, org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),
            thickness = 2)
            
    return frame

In [18]:
#capturing the data from webcam and sending it to the above function and displaying the live results in the screen
video_cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detect = detector(gray_frame, frame)
    
    cv2.imshow("Video", detect)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
video_cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 54ms/step


1/1 [==============================] - 0s 46ms/step
